In [25]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
import os, sys
import pandas as pd
import json
import matplotlib.pyplot as plt

In [27]:
import pandas as pd
import numpy as np
import sqlalchemy as sa
import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta, date

## Main folder

In [28]:
data_folder_path = r"C:\git\CHRIS_SCRIPTS\benchmark\data"

*Move to the data folder*

In [29]:
os.chdir(data_folder_path)

*Print the content of the folder, with the linux `ls` command*

## Supporting functions

In [30]:
def query_arc_alchemy(sql):
    # ODBC connection string
    _url ='mssql+pyodbc:///?odbc_connect=DATABASE%3DARCTIMESERIES%3BDRIVER%3D%7BSQL+Server+Native+Client+11.0%7D%3BSERVER%3DARCSQL%3BTrace%3D0%3BTrusted_Connection%3DYes'
    engine =sa.create_engine(_url, echo=False, convert_unicode=True)
    with engine.connect() as connection:
        result =connection.execute(sql)
        data =result.fetchall()
        column_names =result.keys()
        dataframe = pd.DataFrame.from_records(data, columns=column_names)

        return dataframe

In [31]:
def daterange(date1, date2):
    for n in range(int((date2 - date1).days) + 1):
        yield date1 + timedelta(n)


## Data

In [32]:
Costs = pd.read_csv(r'Costs.csv')
Variables = pd.read_csv(r'Variables.csv')

***

In [33]:
import sys, os
sys.path.append(r'C:\dev\code\gmt-amp\python-analytics-platform')
sys.path.append(r'C:\dev\code\gmt-amp\amp-models')

sys.path.append(r"C:\git\CHRIS_SCRIPTS")

## Transformations

In [34]:
Variables['Days Travel temp'] = Variables['Distances']  \
/ (Costs['Laden - Speed (knots)'].iloc[0] * 24)

In [35]:
choices = [0, 1, 1.3]
conditions = [
    (Variables['Canal Fee'] == 'No'),
    (Variables['Canal Fee'] == 'Suez'),
    (Variables['Canal Fee'] == 'Panama')]
Variables['Add days'] = np.select(conditions, choices, default=9999999)

In [36]:
Variables['Days Travel (one-way inc. transit)'] = Variables['Days Travel temp'] \
+ Variables['Add days']

In [37]:
Variables['Days Travel (Return - inc. time at port)'] = \
                    + Variables['Days Travel (one-way inc. transit)'] * 2 \
                    + Costs['OneWay - Port turnaround time (Load+Discharge) (days)'].iloc[0]

In [38]:
Variables['Boil-off'] =  \
    Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]  \
    * Costs['Laden - Natural boil-off gas (%/day)'].iloc[0] \
    * Variables[ 'Days Travel (Return - inc. time at port)'] 

In [39]:
Variables['Delivered Volume'] =   Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]\
                                - Variables['Boil-off']

In [54]:
choices2 = ['EU', 'EU', 'EU', 'ME', 'ME', 'NEA', 'NEA', 'LatAm', 'EU', 'EU', 'EU', 'EU', 'EU', 'EU','EU']
conditions2 = [
    (Variables['TerminalTo'] == 'UK (Hook)'),
    (Variables['TerminalTo'] == 'UK (Hook)(NBP)'),
    (Variables['TerminalTo'] == 'Belgium (Zee)'),
    (Variables['TerminalTo'] == 'India (Dhambol)'),
    (Variables['TerminalTo'] == 'Middle East (UAE)'),
    (Variables['TerminalTo'] == 'China (Guandong)'),
    (Variables['TerminalTo'] == 'Japan (Tokyo)'),
    (Variables['TerminalTo'] == "Argentina (B'Blanca)"),
    (Variables['TerminalTo'] == 'France (Montoir)'),
    (Variables['TerminalTo'] == 'Spain (Barcelona)'),
    (Variables['TerminalTo'] == 'Spain (Barcelona)(PVB)'),
    (Variables['TerminalTo'] == 'Italy (Rovigo)'),
    (Variables['TerminalTo'] == 'Italy (Rovigo)(PSV)'),
    (Variables['TerminalTo'] == 'Netherlands (Gate)'),
    (Variables['TerminalTo'] == 'Netherlands (Gate)(TTF)')]

Variables['RegionCode'] = np.select(conditions2, choices2, default=9999999)

In [41]:
conditions3 = [
    (Variables['TerminalTo'] == "Argentina (B'Blanca)"),
    (Variables['TerminalTo'] == 'Belgium (Zee)'),
    (Variables['TerminalTo'] == 'China (Guandong)'),
    (Variables['TerminalTo'] == 'France (Montoir)'),
    (Variables['TerminalTo'] == 'India (Dhambol)'),
    (Variables['TerminalTo'] == 'Italy (Rovigo)'),
    (Variables['TerminalTo'] == 'Italy (Rovigo)(PSV)'),
    (Variables['TerminalTo'] == 'Japan (Tokyo)'),
    (Variables['TerminalTo'] == 'Middle East (UAE)'),
    (Variables['TerminalTo'] == 'Netherlands (Gate)'),
    (Variables['TerminalTo'] == 'Netherlands (Gate)(TTF)'),
    (Variables['TerminalTo'] == 'Spain (Barcelona)'),
    (Variables['TerminalTo'] == 'Spain (Barcelona)(PVB)'),
    (Variables['TerminalTo'] == 'UK (Hook)'),
    (Variables['TerminalTo'] == 'UK (Hook)(NBP)')]

choices3 = ['LNG_H_DES_Lat_Am_MID_M_USD', 
            'G_H_Zeebrugge-Hub_Pnc', 
            'LNG_JKM_M_USD', 
            'G_H_PEG-Nord_EUR',
            'LNG_DES_ME_W_INDIA_M_USD',
            'LNG_H_DES_Med_MID_M_USD',
            "G_H_PSV_EUR", 
            'LNG_JKM_M_USD', 
            'LNG_DES_ME_W_INDIA_M_USD', 
            'LNG_H_DES_NWE_MID_M_USD',
            'G_H_TTF_95%_EUR',
            'LNG_H_DES_Med_MID_M_USD',
            'G_H_AOC_EUR', 
            'LNG_H_DES_NWE_MID_M_USD',
            'G_H_NBP_95%_USD']
Variables['CurveName'] = np.select(conditions3, choices3, default=9999999)

In [42]:
conditions4 = [        
    (Variables['TerminalFrom'] == 'Sabine'),
    (Variables['TerminalFrom'] == 'Cove point'),
    (Variables['TerminalFrom'] == 'Nigeria'),
    (Variables['TerminalFrom'] == 'Angola'),
    (Variables['TerminalFrom'] == 'Qatar'),
    (Variables['TerminalFrom'] == 'Algeria'),
    (Variables['TerminalFrom'] == 'Yamal'),
    (Variables['TerminalFrom'] == 'Wheatstone'),
    (Variables['TerminalFrom'] == 'Gladstone'),
    (Variables['TerminalFrom'] == 'T&T')]
choices4 = ['G_NYM_HH_M_115%_USD', 
            'G_NYM_HH_M_115%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD',
            'LNG_JKM_M_0%_USD']    
Variables['OriginPriceCurve'] = np.select(conditions4, choices4, default=9999999)  

***

## Constants and mappings

In [43]:
from gmt.dna.datagateway.data_adapter_factory import DataAdapterFactory
from gmt.dna.amp.models.gas.net_back.net_back import NetBackSymbols
import nest_asyncio
nest_asyncio.apply()

In [44]:
weekend = [6, 7]

start_date = date(2021, 1, 14)
end_date = date(2021, 1, 14)

In [45]:
def _get_one_arc_curve(curve, _t, _from_date,
                       _to_date) -> pd.DataFrame:
    data_adaptor = DataAdapterFactory.create("arc", env='prod')
    curve_df = data_adaptor.get_forward_curve(
        curve, _t).resample('M').mean().loc[_from_date:_to_date]
    curve_df.index = [date(t.year, t.month, 1) for t in curve_df.index]
    curve_df = curve_df.to_frame()
    curve_df.reset_index(inplace=True)
    curve_df.columns = [NetBackSymbols.SYM_POINTDATE, NetBackSymbols.SYM_POINTVALUE]
    curve_df[NetBackSymbols.SYM_CURVENAME] = curve
    return curve_df

arc_curves = ['G_H_NBP.USD',
 'G_H_Zeebrugge-Hub.Pnc',
 'LNG_JKM_M.USD',
 'G_H_TRS.EUR',
 'LNG_DES_ME_W_INDIA_M.USD',
 'LNG_H_DES_Med_MID_M.USD',
 'LNG_H_DES_JKM_MID_M.USD',
 'LNG_H_DES_NWE_MID_M.USD',
 'G_H_TTF.EUR',
 'G_H_CEGH.EUR',
 'G_H_AOC.EUR',
 'G_H_PEG-Nord.EUR',
 'G_H_PSV.EUR',
 'G_NYM_HH_M.USD',
 'LNG_H_DES_Lat_Am_MID_M.USD',
 'FX_curve_cross_EUR.USD',
 'FX_curve_input_USD.GBP',
 'FREIGHT_H_LNG_M_RolledOver.USD']

In [46]:
start_date = date(2021, 1, 14)
end_date = date(2021, 1, 14)

In [47]:
print('Started computing')
t_start = datetime.now()

Netback_short_all_t = pd.DataFrame()
Netback_all_t = pd.DataFrame()

n = len(list(daterange(start_date, end_date)))
print(f'There are {n} days to process.\nLeft:\n', end=' ')

for dt in list(daterange(start_date, end_date)):    
    print(f'{n-1}', end='; '); n-=1
    if dt.isoweekday() in weekend:
        continue  
    
    print(f'{dt}', end='; '); 

    today = dt.day
    if today > 20:
        from_date = dt + pd.DateOffset(months=2)
        to_date = dt + pd.DateOffset(months=2 ,years=2)    
    else:
        from_date = dt + pd.DateOffset(months=1)
        to_date = dt + pd.DateOffset(months=1 ,years=2)


    query_str_1 = f"""
                    SELECT
                      d.[CurveName],
                      datefromparts(year(min(d.PointDate)), month(min(d.PointDate)), 1) as PointDate,
                      AVG(d.[PointValue]) as PointValue
                    FROM [arc_ts_data].[PriceCurveData] d
                    WHERE d.EffectiveDate = '{dt.strftime("%Y-%m-%d")}' and [CurveName] in 
                    ('G_H_NBP.USD', 'G_H_Zeebrugge-Hub.Pnc','LNG_JKM_M.USD', 'G_H_TRS.EUR', 
                    'LNG_DES_ME_W_INDIA_M.USD', 'LNG_H_DES_Med_MID_M.USD', 'LNG_H_DES_JKM_MID_M.USD', 
                    'LNG_H_DES_NWE_MID_M.USD', 'G_H_TTF.EUR', 'G_H_CEGH.EUR', 'G_H_AOC.EUR', 'G_H_PEG-Nord.EUR', 
                    'G_H_PSV.EUR','G_NYM_HH_M.USD' , 'LNG_H_DES_Lat_Am_MID_M.USD', 'FX_curve_cross_EUR.USD', 
                    'FX_curve_input_USD.GBP', 'FREIGHT_H_LNG_M_RolledOver.USD')  
                    and d.PointDate >= '{from_date.strftime("%Y-%m-01")}' and d.PointDate < '{to_date.strftime("%Y-%m-01")}'
                    group by d.CurveId, d.CurveName, d.EffectiveDate, 12 * year(d.PointDate) + month(d.PointDate)
                    order by d.CurveID
                    """    
    #------------------------------------------------------Prices---------------------------------------------------------------
#     Prices = query_arc_alchemy(query_str_1)    
    
    Prices = pd.concat([
    _get_one_arc_curve(curve, dt, from_date, to_date)
    for curve in arc_curves
],
    axis=0,
    sort=False)    
    

    Prices['CurveName'] = Prices['CurveName'].str.replace('.', '_')    
    Prices['PointDate_Origin'] = pd.to_datetime(Prices['PointDate'])
    Prices['EffectiveDate'] = dt
    Prices['EffectiveDate'] = pd.to_datetime(Prices['EffectiveDate'])
    Prices['PointDate_Destination'] = Prices['PointDate_Origin']    
    
    columns = ['CurveName', 'PointValue', 'PointDate_Origin', 'EffectiveDate','PointDate_Destination']
    Prices = Prices[columns]    
    
    
    
    mapper = Prices.query('CurveName == "FX_curve_cross_EUR_USD"').set_index(['PointDate_Origin'])['PointValue']    
    Prices['FX_curve_cross_EUR_USD'] = Prices['PointDate_Origin'].map(mapper)    
    Prices['FX_curve_cross_EUR_USD'] = Prices['FX_curve_cross_EUR_USD'].fillna(method='ffill')    
    
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("_EUR", case=False),
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') / Prices['FX_curve_cross_EUR_USD'].apply(pd.to_numeric, downcast='float') / 3.412 * 0.95,
                                    Prices['PointValue'])
    
    mapper = Prices.query('CurveName == "FX_curve_input_USD_GBP"').set_index(['PointDate_Origin'])['PointValue']
    Prices['FX_curve_input_USD_GBP'] = Prices['PointDate_Origin'].map(mapper)    
    Prices['FX_curve_input_USD_GBP'] = Prices['FX_curve_input_USD_GBP'].fillna(method='ffill')
    
        
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("_Pnc", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') / Prices['FX_curve_input_USD_GBP'].apply(pd.to_numeric, downcast='float') / 10, 
                                    Prices['PointValue'])
    
    
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_NYM_HH_M_USD", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 1.15, 
                                    Prices['PointValue'])
    
    Prices['CurveName'] = np.where(Prices['CurveName'] == 'G_NYM_HH_M_USD',
                                   'G_NYM_HH_M_115%_USD',
                                   Prices['CurveName'])

    
    #INCREASES THE NBP PRICE BY 15% IN-LINE WITH CHENIERE CONTRACTS
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_H_NBP_USD", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 0.95, 
                                    Prices['PointValue'])
        
    Prices['CurveName'] = np.where((Prices['CurveName'] == 'G_H_NBP_USD'),
                                   'G_H_NBP_95%_USD',
                                   Prices['CurveName'])
    
    
    

    #INCREASES THE NBP PRICE BY 15% IN-LINE WITH CHENIERE CONTRACTS
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_H_TTF_EUR", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 0.95, 
                                    Prices['PointValue'])
    
    Prices['CurveName'] = np.where(
                                    (Prices['CurveName'] == 'G_H_TTF_EUR'),
                                    'G_H_TTF_95%_EUR',
                                    Prices['CurveName']
    )

    
    
    #DECREASES THE JKM PRICE BY 100% 
    Prices['PointValue'] = np.where(Prices['CurveName'].str.contains("G_H_CEGH_EUR", case=False), 
                                    Prices['PointValue'].apply(pd.to_numeric, downcast='float') * 0, 
                                    Prices['PointValue'])
    
    Prices['CurveName'] = np.where(Prices['CurveName'] == 'G_H_CEGH_EUR',
                                   'LNG_JKM_M_0%_USD',
                                   Prices['CurveName'])


    


    
    
    #---------------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------vars_ts_df - the same as vars_ts_df ------------------------------
    #---------------------------------------------------------------------------------------------------------------------------
    date_rng = pd.date_range(start=dt, freq='MS', periods=12, name='Dates')
    combining_date = []
    for d in date_rng:
        Variables['PointDate_Origin'] = d
        combining_date.append(Variables.copy())      
    vars_ts_df = pd.concat(combining_date)    
    
    

    
    mapper = Prices.query('CurveName == "FREIGHT_H_LNG_M_RolledOver_USD"').set_index(['PointDate_Origin'])['PointValue']
    
    vars_ts_df['Shipping Cost'] = np.where(
        vars_ts_df['Days Travel (Return - inc. time at port)'] > 30,        
        
       (((vars_ts_df['PointDate_Origin'] + pd.DateOffset(months=1)).map(mapper) 
        + vars_ts_df['PointDate_Origin'].map(mapper)) / 2),         
        
        vars_ts_df['PointDate_Origin'].map(mapper)    
    )    
    
    
    
    vars_ts_df[['Regas Costs', 'Port Charges', 'Boil-off', 'Delivered Volume', 'Shipping Cost']] = vars_ts_df[
        ['Regas Costs', 'Port Charges', 'Boil-off', 'Delivered Volume', 'Shipping Cost']].apply(pd.to_numeric,
                                                                                                downcast='float')
    
    
    vars_ts_df['ShippingCostOutput'] = (
        vars_ts_df['Shipping Cost'] *
        vars_ts_df['Days Travel (Return - inc. time at port)'] /
        Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]
    )
    
    
    
    #                  
    #---------------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------Netback-----------------------------------------------------
    #---------------------------------------------------------------------------------------------------------------------------
    
    Netback = vars_ts_df.merge(
        Prices[['PointDate_Origin','CurveName','EffectiveDate']], how='inner', on=['PointDate_Origin', 'CurveName'])
    
    Netback = Netback.merge(Prices[['PointDate_Origin', 'CurveName','PointValue']],
                                              how='inner', 
                                              right_on=['PointDate_Origin', 'CurveName'],
                                              left_on=['PointDate_Origin', 'OriginPriceCurve'])    
    
    Netback = Netback.drop(columns=['CurveName_y'])   
    Netback.rename(columns={'CurveName_x':'CurveName_Destination','PointValue':'PointValue_Origin'}, inplace=True)     
    
    
    
    
    
    Netback['PointDate_Destination'] = Netback['PointDate_Origin']   
    

    
    Netback['PointDate_Destination'] = np.where(                        
                    Netback['Days Travel (Return - inc. time at port)'] > 30, 
                    Netback['PointDate_Destination'] + pd.DateOffset(months=1), 
                    Netback['PointDate_Destination']
    )    
    
    
    Netback = Netback.merge(Prices[['PointDate_Destination','CurveName','PointValue']], 
                                right_on=['PointDate_Destination', 'CurveName'],
                                left_on=['PointDate_Destination', 'CurveName_Destination'])
    
    
    
    Netback.rename(columns={'PointValue':'PointValue_Destination'}, inplace=True)
    
    Netback = Netback.drop(columns=['CurveName'])
    
    
    
    Netback[['Boil-off', 'PointValue_Destination','PointValue_Origin']] = \
    Netback[['Boil-off', 'PointValue_Destination', 'PointValue_Origin']].apply(pd.to_numeric,downcast='float')
    
    
    
    Netback['BoilOffOutput'] = (
            Netback['PointValue_Destination'] * Netback['Boil-off'] / Costs['Loaded - Gas cargo (mmbtu)'].iloc[0])
    
    
    
    
    Netback['FuelOutput'] = np.where(
        
                            Netback['TerminalFrom'] == 'Yamal', 

                            ((Costs['OneWay - Port turnaround time (Load+Discharge) (days)'].iloc[0] *
                              Costs['Fuel consumption in port (tonnes/day)'].iloc[0] *
                              Costs['USD/tonne - HFO'].iloc[0]) + \
                             (Costs['USD/tonne - HFO'].iloc[0] *
                              Costs['Fuel oil consumption (tonnes/day)'].iloc[0] * 2 * Netback[
                                                    'Days Travel (one-way inc. transit)'])) /
                                              Costs['Loaded - Gas cargo (mmbtu)'].iloc[0], 
        
                            0.06)
    
    
    
    
    Netback['CanalCostsOutput'] = np.where(
                            Netback['Canal Fee'] == 'Panama', (
                                
                                
                            Costs['Panama - Total ($)'].iloc[0] / Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]), 
                                                    np.where(Netback['Canal Fee'] == 'Suez', (
                                                            Costs['Suez - Total ($)'].iloc[0] /
                                                            Costs['Loaded - Gas cargo (mmbtu)'].iloc[0]), 
                                                             
                                                             0))
    
    
    
    Netback['PortCostsOutput'] = (Netback['Port Charges'] / Costs['Loaded - Gas cargo (mmbtu)'].iloc[0])
    
    
    
    Netback['TotalCostIncluding(Regas&Port)'] = Netback['ShippingCostOutput'] + \
                                                         Netback['BoilOffOutput'] + \
                                                         Netback['FuelOutput'] + \
                                                         Netback['CanalCostsOutput'] + \
                                                         Netback['PortCostsOutput'] + \
                                                         Netback['Regas Costs']
    
    
    
    Netback['TotalCostIncluding(Regas)'] = Netback['ShippingCostOutput'] + \
                                                    Netback['BoilOffOutput'] + \
                                                    Netback['FuelOutput'] + \
                                                    Netback['CanalCostsOutput'] + \
                                                    Netback['Regas Costs']
    
    
    
    Netback['TotalCostIncluding(Port)'] = Netback['ShippingCostOutput'] + \
                                                   Netback['BoilOffOutput'] + \
                                                   Netback['FuelOutput'] + \
                                                   Netback['CanalCostsOutput'] + \
                                                   Netback['PortCostsOutput']
    
    
    Netback['TotalCost'] = Netback['ShippingCostOutput'] + \
                                    Netback['BoilOffOutput'] + \
                                    Netback['FuelOutput'] + \
                                    Netback['CanalCostsOutput']
    
    Netback['TotalCost(Sunk Shiping)'] = Netback['ShippingCostOutput'] + \
                                Netback['BoilOffOutput'] + \
                                Netback['FuelOutput'] + \
                                Netback['CanalCostsOutput']    
    
    
    Netback['Variable_Shipping_Roundtrip'] = (Netback['PointValue_Destination'] - 
                                              Netback['BoilOffOutput'] - 
                                              Netback['FuelOutput'] - 
                                              Netback['PortCostsOutput'] - 
                                              Netback['PointValue_Origin'])

    
    
    Netback['Variable_Shipping_Single'] = (Netback['PointValue_Destination'] - 
                                           0.5*Netback['BoilOffOutput'] -  
                                           Netback['FuelOutput'] - 
                                           Netback['PortCostsOutput'] - 
                                           Netback['PointValue_Origin'])
    #-----------------------------------------------STOPPED-----------------------------------------------------------------------
    
    Netback['Full_Shipping_Rountrip'] = (Netback['PointValue_Destination'] - 
                                         Netback['TotalCostIncluding(Port)'] - 
                                         Netback['PointValue_Origin'])
    #-----------------------------------------------STOPPED-----------------------------------------------------------------------

    #---------------------------------------------------------------------------------------------------------------------------
    #------------------------------------------------------Netback--------------------------------------------------------------
    #---------------------------------------------------------------------------------------------------------------------------
    
    Netback_all_t = Netback_all_t.append(Netback, ignore_index = True )
        
    mask = ['EffectiveDate', 
            'PointDate_Origin', 
            'PointDate_Destination',
            'RegionFrom', 
            'TerminalFrom', 
            'TerminalTo', 
            'TotalCost', 
            'TotalCostIncluding(Regas&Port)',                    
            'TotalCostIncluding(Regas)',
            'TotalCostIncluding(Port)',
            'Full_Shipping_Rountrip',
            'Variable_Shipping_Single',
            'Variable_Shipping_Roundtrip']
    
    Netback_short = Netback[mask]

    Netback_short_all_t = Netback_short_all_t.append(Netback_short, ignore_index = True )

    del vars_ts_df
    
t_end = datetime.now()
print(f'Main took {(t_end - t_start).seconds} seconds')

Started computing
There are 1 days to process.
Left:
 0; 2021-01-14; Main took 4 seconds


## Store results

In [48]:
root_path = r'C:\git\CHRIS_SCRIPTS\naked_script_all_t9'

In [49]:
path_net_back_all_t_chris = os.path.join(root_path, 'Netback.csv')
path_net_back_short_all_t_chris = os.path.join(root_path, 'Netback_narrow.csv')

Netback_all_t.to_csv(path_net_back_all_t_chris, index=False)
Netback_short_all_t.to_csv(path_net_back_short_all_t_chris, index=False)

In [50]:
os.chdir(root_path)

In [51]:
print('Processing by TerminalFrom ...')

columns = ['EffectiveDate', 'PointDate_Origin', 'PointDate_Destination', 'TerminalTo', 
           'Variable_Shipping_Roundtrip']
netback_by_terminal_from = Netback_all_t[columns].groupby(Netback_all_t['TerminalFrom'])

for terminal_from, group in netback_by_terminal_from:
    print(f'Processing TerminalFrom {terminal_from}')

    pivoted = group.pivot_table(
        'Variable_Shipping_Roundtrip', ['PointDate_Destination', 'EffectiveDate'], 'TerminalTo'
    )
    pivoted.dropna(inplace=True)
    pivoted['Netherlands (Gate)-Japan (Tokyo)'] = pivoted['Netherlands (Gate)']\
    - pivoted['Japan (Tokyo)']
    pivoted['UK (Hook)-Japan (Tokyo)'] = pivoted['UK (Hook)'] - pivoted['Japan (Tokyo)']

    csv_file = f'{terminal_from}_Pivot_VarShip.csv'
    pivoted.to_csv(csv_file)

    print(f'Data for TerminalFrom {terminal_from} saved to {csv_file}')

print('Processing by TerminalFrom finished')
#==========================================================================
print('Processing by TerminalFrom ...')

columns = ['EffectiveDate', 'PointDate_Origin', 'PointDate_Destination', 'TerminalTo', 'Full_Shipping_Rountrip']
netback_by_terminal_from = Netback_all_t[columns].groupby(Netback_all_t['TerminalFrom'])

for terminal_from, group in netback_by_terminal_from:
    print(f'Processing TerminalFrom {terminal_from}')

    pivoted = group.pivot_table(
        'Full_Shipping_Rountrip', ['PointDate_Destination', 'EffectiveDate'], 'TerminalTo'
    )
    pivoted.dropna(inplace=True)
    pivoted['Netherlands (Gate)-Japan (Tokyo)'] = pivoted['Netherlands (Gate)'] - pivoted['Japan (Tokyo)']
    pivoted['UK (Hook)-Japan (Tokyo)'] = pivoted['UK (Hook)'] - pivoted['Japan (Tokyo)']

    csv_file = f'{terminal_from}_Pivot_FullShip.csv'
    pivoted.to_csv(csv_file)

    print(f'Data for TerminalFrom {terminal_from} saved to {csv_file}')

print('Processing by TerminalFrom finished')

Processing by TerminalFrom ...
Processing TerminalFrom Algeria
Data for TerminalFrom Algeria saved to Algeria_Pivot_VarShip.csv
Processing TerminalFrom Angola
Data for TerminalFrom Angola saved to Angola_Pivot_VarShip.csv
Processing TerminalFrom Cove point
Data for TerminalFrom Cove point saved to Cove point_Pivot_VarShip.csv
Processing TerminalFrom Gladstone
Data for TerminalFrom Gladstone saved to Gladstone_Pivot_VarShip.csv
Processing TerminalFrom Nigeria
Data for TerminalFrom Nigeria saved to Nigeria_Pivot_VarShip.csv
Processing TerminalFrom Qatar
Data for TerminalFrom Qatar saved to Qatar_Pivot_VarShip.csv
Processing TerminalFrom Sabine
Data for TerminalFrom Sabine saved to Sabine_Pivot_VarShip.csv
Processing TerminalFrom T&T
Data for TerminalFrom T&T saved to T&T_Pivot_VarShip.csv
Processing TerminalFrom Wheatstone
Data for TerminalFrom Wheatstone saved to Wheatstone_Pivot_VarShip.csv
Processing TerminalFrom Yamal
Data for TerminalFrom Yamal saved to Yamal_Pivot_VarShip.csv
Proce